In [2]:
import numpy as np
import os 
import json
import chromadb
from core.cfg import embedding_function
from core.cfg import COLLECTION_PATH, root_img_path
from core.utilies import get_files_path,plot_chromadb_results
HNSW_SPACE = "hnsw:space"

__doc__=[
    "query image in database with chromadb"
]
files_path = get_files_path(path=root_img_path)    
    
def __get_single_image_embedding(image : np.ndarray):
    """
        * Return feature of query image
    """
    embedding = embedding_function._encode_image(image=image)
    return np.array(embedding)

def __search(image_path, collection, n_results):
    query_image = Image.open(image_path)
    query_embedding = __get_single_image_embedding(query_image)
    results = collection.query(
            query_embeddings=[query_embedding.tolist()],
            n_results=n_results 
    )
    return results

## Collection Loading and Features Loading

In [3]:
def load_collection(client,collection_name,metadata,load_path):
    """
        Collection loading: Return Collection
        * Collection Building
        * Features embeddings loading
        * ids loading
        * mapping_paths loading
    """
    
    # Load the ids from the .json file
    ids_path = os.path.join(load_path, 'ids.json')
    with open(ids_path, 'r') as ids_file:
        ids = json.load(ids_file)

    # Load the paths from the .json file
    mapping_paths = os.path.join(load_path, 'mapping_paths.json')
    with open(mapping_paths, 'r') as paths_file:
        mapping = json.load(paths_file)

    # Load the embeddings from the .npy file
    embeddings_path = os.path.join(load_path, 'embeddings.npy')
    embeddings = np.load(embeddings_path, allow_pickle=True)

    # Get or create the collection in ChromaDB
    #collection = client.get_or_create_collection(name=collection_name)
    collection = client.get_or_create_collection( name = collection_name,
                                                        metadata={HNSW_SPACE: metadata})
    # Add the embeddings and ids back to the collection
    collection.add(
        embeddings=embeddings.tolist(), 
        ids = ids)

    return collection, mapping

## Query by l2

In [3]:
client = chromadb.Client()

ids_path = os.path.join(COLLECTION_PATH, 'ids.json')
with open(ids_path, 'r') as ids_file:
    ids = json.load(ids_file)

# Load the paths from the .json file
mapping_paths = os.path.join(COLLECTION_PATH, 'mapping_paths.json')
with open(mapping_paths, 'r') as paths_file:
    mapping = json.load(paths_file)

# Load the embeddings from the .npy file
embeddings_path = os.path.join(COLLECTION_PATH, 'embeddings.npy')
embeddings = np.load(embeddings_path, allow_pickle=True)

In [ ]:
l2_collection = client.get_or_create_collection( name = 'l2_collection',
                                                        metadata={HNSW_SPACE: "l2"})
l2_collection.add(
    embeddings=embeddings.tolist(), 
    ids = ids)

In [ ]:
test_path = r'data/test/African_crocodile/n01697457_18534.JPEG'
results = __search(image_path=test_path, collection = l2_collection, n_results=5)
plot_chromadb_results(test_path, mapping, results)

## Query by Cosine

In [ ]:
cosine_collection = client.get_or_create_collection( name = 'cosine_collection',
                                                     metadata={HNSW_SPACE: "cosine"})
cosine_collection.add(
    embeddings=embeddings.tolist(), 
    ids = ids)

In [ ]:
test_path = r'data/test/African_crocodile/n01697457_18534.JPEG'
results = __search(image_path=test_path, collection = cosine_collection, n_results=5)
plot_chromadb_results(test_path, mapping, results)